In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('MoviesOnStreamingPlatforms.csv')
df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       16744 non-null  int64  
 1   ID               16744 non-null  int64  
 2   Title            16744 non-null  object 
 3   Year             16744 non-null  int64  
 4   Age              7354 non-null   object 
 5   IMDb             16173 non-null  float64
 6   Rotten Tomatoes  5158 non-null   object 
 7   Netflix          16744 non-null  int64  
 8   Hulu             16744 non-null  int64  
 9   Prime Video      16744 non-null  int64  
 10  Disney+          16744 non-null  int64  
 11  Type             16744 non-null  int64  
 12  Directors        16018 non-null  object 
 13  Genres           16469 non-null  object 
 14  Country          16309 non-null  object 
 15  Language         16130 non-null  object 
 16  Runtime          16152 non-null  float64
dtypes: float64(2

In [4]:
df.columns

Index(['Unnamed: 0', 'ID', 'Title', 'Year', 'Age', 'IMDb', 'Rotten Tomatoes',
       'Netflix', 'Hulu', 'Prime Video', 'Disney+', 'Type', 'Directors',
       'Genres', 'Country', 'Language', 'Runtime'],
      dtype='object')

In [5]:
df = df.drop(columns=['Unnamed: 0'])

In [6]:
df = df.drop(columns=['Type','ID','Rotten Tomatoes'])

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        16744 non-null  object 
 1   Year         16744 non-null  int64  
 2   Age          7354 non-null   object 
 3   IMDb         16173 non-null  float64
 4   Netflix      16744 non-null  int64  
 5   Hulu         16744 non-null  int64  
 6   Prime Video  16744 non-null  int64  
 7   Disney+      16744 non-null  int64  
 8   Directors    16018 non-null  object 
 9   Genres       16469 non-null  object 
 10  Country      16309 non-null  object 
 11  Language     16130 non-null  object 
 12  Runtime      16152 non-null  float64
dtypes: float64(2), int64(5), object(6)
memory usage: 1.7+ MB


In [8]:
cat_cols = ["Directors", "Genres", "Country", "Language"]

df[cat_cols] = df[cat_cols].fillna("Unknown")

In [9]:
df['Runtime'] = df['Runtime'].fillna(df['Runtime'].median())

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16744 entries, 0 to 16743
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        16744 non-null  object 
 1   Year         16744 non-null  int64  
 2   Age          7354 non-null   object 
 3   IMDb         16173 non-null  float64
 4   Netflix      16744 non-null  int64  
 5   Hulu         16744 non-null  int64  
 6   Prime Video  16744 non-null  int64  
 7   Disney+      16744 non-null  int64  
 8   Directors    16744 non-null  object 
 9   Genres       16744 non-null  object 
 10  Country      16744 non-null  object 
 11  Language     16744 non-null  object 
 12  Runtime      16744 non-null  float64
dtypes: float64(2), int64(5), object(6)
memory usage: 1.7+ MB


In [11]:
df = df.dropna(axis=0,subset=['Age','IMDb'])

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7316 entries, 0 to 16740
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Title        7316 non-null   object 
 1   Year         7316 non-null   int64  
 2   Age          7316 non-null   object 
 3   IMDb         7316 non-null   float64
 4   Netflix      7316 non-null   int64  
 5   Hulu         7316 non-null   int64  
 6   Prime Video  7316 non-null   int64  
 7   Disney+      7316 non-null   int64  
 8   Directors    7316 non-null   object 
 9   Genres       7316 non-null   object 
 10  Country      7316 non-null   object 
 11  Language     7316 non-null   object 
 12  Runtime      7316 non-null   float64
dtypes: float64(2), int64(5), object(6)
memory usage: 800.2+ KB


In [14]:
df.duplicated().any()

False

In [20]:
def run_time(x):
    if x["Runtime"] == 0:
        return "Unknown"
    elif x["Runtime"] <= 90:
        return "Short Movie"
    elif x["Runtime"] <= 200:
        return "Long Movie"
    else:
        return "Series"

df["Runtime_Category"] = df.apply(run_time,axis=1)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7316 entries, 0 to 16740
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             7316 non-null   object 
 1   Year              7316 non-null   int64  
 2   Age               7316 non-null   object 
 3   IMDb              7316 non-null   float64
 4   Netflix           7316 non-null   int64  
 5   Hulu              7316 non-null   int64  
 6   Prime Video       7316 non-null   int64  
 7   Disney+           7316 non-null   int64  
 8   Directors         7316 non-null   object 
 9   Genres            7316 non-null   object 
 10  Country           7316 non-null   object 
 11  Language          7316 non-null   object 
 12  Runtime           7316 non-null   float64
 13  Runtime_Category  7316 non-null   object 
dtypes: float64(2), int64(5), object(7)
memory usage: 857.3+ KB


In [31]:
plt_df = pd.melt(df,id_vars=['Title','Age','IMDb'],value_vars=['Netflix','Hulu','Prime Video','Disney+'],
        var_name='Streaming Platform',value_name='Availablity')

plt_df

,Title,Age,IMDb,Streaming Platform,Availablity
0,Inception,13+,8.8,Netflix,1
1,The Matrix,18+,8.7,Netflix,1
2,Avengers: Infinity War,13+,8.5,Netflix,1
3,Back to the Future,7+,8.5,Netflix,1
4,"The Good, the Bad and the Ugly",18+,8.8,Netflix,1
...,...,...,...,...,...
29259,Richie Rich's Christmas Wish,all,4.1,Disney+,1
29260,The Jungle Book: Mowgli's Story,all,4.2,Disney+,1
29261,The Bears and I,all,6.2,Disney+,1
29262,Whispers: An Elephant's Tale,all,5.0,Disney+,1


In [32]:
plt_df = plt_df.loc[plt_df['Availablity'] == 1]

In [33]:
plt_df = plt_df.sort_values(by=['Title']).reset_index(drop=True)

In [34]:
plt_df

,Title,Age,IMDb,Streaming Platform,Availablity
0,#IMomSoHard Live,18+,5.2,Prime Video,1
1,#LoveSwag,13+,5.4,Prime Video,1
2,#cats_the_mewvie,16+,5.2,Netflix,1
3,$elfie Shootout,16+,3.4,Prime Video,1
4,...And Your Name Is Jonah,7+,7.1,Prime Video,1
...,...,...,...,...,...
7703,Æon Flux,13+,5.4,Hulu,1
7704,İncir Reçeli 2,13+,6.1,Netflix,1
7705,审死官,7+,6.9,Netflix,1
7706,审死官,7+,6.9,Prime Video,1
